<a href="https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/Face_depixelizer_ttmai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Face Depixelizer</font></b>

Given a low-resolution input image, Face Depixelizer searches the outputs of a generative model (here, StyleGAN) for high-resolution images that are perceptually realistic and downscale correctly.

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [PULSE](https://github.com/adamian98/pulse)

Article: [PULSE: Self-Supervised Photo Upsampling via Latent Space Exploration of Generative Models](https://arxiv.org/abs/2003.03808)

Creators: **[Alex Damian](https://github.com/adamian98), [Sachit Menon](mailto:sachit.menon@duke.edu).**

<b><font color="black" size="+2">Colab created by:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).

---
##### <font color='red'>Επειδή χρησιμοποιείται το google drive μου για να κατεβάσει τα αρχεία των εκπαιδευμένων μοντέλων, υπάρχει περίπτωση να φτάσουμε το όριο των κλήσεων προς το google drive για σήμερα. Αν δεις τα errors:  "Google Drive Quota Exceeded" ή "No such file or directory: '/content/pulse/runs/face.png'" Δοκίμασε να ξανατρέξεις το notebook αργότερα.</font>



<b><font color="black" size="+2">Αυτό το colab notebook έχει αλλαγές απο:</font></b>

GitHub: [@fzantalis](https://github.com/fzantalis)

Σημείωση: Ο κώδικας έχει κρυφτεί σε αυτό το notebook. Οπότε μην ψάχνετε για γκρι blocks. Κατα τ άλλα, πατάτε κανονικά τα κουμπάκια με την σειρά.

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+2">Εισαγωγή βιβλιοθηκών</font></b>
#@markdown ---
import torch
import torchvision
from pathlib import Path
if not Path("PULSE.py").exists():
  if Path("pulse").exists():
    %cd /content/pulse
  else:
    !git clone https://github.com/adamian98/pulse
    %cd /content/pulse
    !mkdir input/
    toPIL = torchvision.transforms.ToPILImage()
    toTensor = torchvision.transforms.ToTensor()
    from bicubic import BicubicDownSample
    D = BicubicDownSample(factor=1)
    !sed -i 's+https://drive.google.com/uc?id=1huhv8PYpNNKbGCLOaYUjOgR1pY5pmbJx+https://drive.google.com/u/0/uc?id=1jyQC-xs_4coRappR95ys4-uqUtiVXXtl+g' align_face.py
    !rm -f run.py
    !rm -f PULSE.py
    !gdown https://drive.google.com/u/0/uc?id=1N4psZ7qSNiFqaliBtWBBqwLig8GFB304
    !gdown https://drive.google.com/u/0/uc?id=1EFUmsxKDa9NS4L4ukbAt1LrxWk3mMVIl

import os
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from PULSE import PULSE
from google.colab import files
from bicubic import BicubicDownSample
from IPython import display
from IPython.display import display
from IPython.display import clear_output
import numpy as np
from drive import open_url
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline




In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3">Πάτα το και φύγαμε!</font></b>
#@markdown *Μόλις πατήσεις αυτό το κουμπάκι, θα σου φορτώσει παρακάτω η φόρμα για να ανεβάσεις την δική σου φωτογραφία. Η φωτογραφία πρέπει να είναι τετράγωνη (πχ: 500x500 pixels) και θα πρέπει να περιέχει μόνο ένα πρόσωπο, το οποίο θα κοιτάει ευθεία στην κάμερα. Παράδειγμα:*

#@markdown ![example](https://github.com/tg-bomze/Face-Depixelizer/raw/master/example.jpg)

#@markdown *Μπορείς να κόψεις τις φωτογραφίες σου με αυτό το online tool.[ΕΔΩ](https://www.iloveimg.com/crop-image)*
#@markdown ##### *Η απλά με το Paint στον υπολογιστή σου*
#@markdown ## Βασικές Ρυθμίσεις:
#@markdown ##### *Αυτό το πρόγραμμα λειτουργεί με 2 τρόπους.* 
#@markdown ##### *1) Μπορείς να ανεβάσεις μία καθαρή κανονική φωτογραφία (επιλέγοντας το **"clean photo"**) και το πρόγραμμα θα πιξελιάσει μόνο του την φωτογραφία και στην συνέχεια θα προσπαθήσει να την 'καθαρίσει'.*
#@markdown ##### *2) Μπορείς να ανεβάσεις κατευθείαν μια πιξελιασμένη ή θολή φωτογράφια (επιλέγοντας το **"pixelized photo"**).* 

upload_photo = 'clean photo' #@param ["pixelized photo", "clean photo"]



!rm -rf /content/pulse/input/face.png

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
os.rename(fn, fn.replace(" ", ""))
fn = fn.replace(" ", "")

if(len(uploaded.keys())!=1): raise Exception("You need to upload only one image.")

face = Image.open(fn)
face = face.resize((1024, 1024), Image.ANTIALIAS)
face = face.convert('RGB')
face_name = 'face.png'
face.save(face_name)
if upload_photo == "pixelized photo":
  %cp $face_name /content/pulse/input/
elif upload_photo == "clean photo":
  %mkdir -p /content/pulse/realpics
  %mv $face_name /content/pulse/realpics/
  !python align_face.py
  %mv /content/pulse/input/face* /content/pulse/input/$face_name



#@markdown ---
#@markdown ## Ρυθμίσεις για προχωρημένους:
#@markdown ##### *Άλλαξε τις παρακάτω τιμές μόνο αν θες να πετύχεις ένα διαφορετικό ή πιο ακριβές αποτέλεσμα.* 
#@markdown #### **DEFAULT** *variables*:

seed = 100 #@param {type:"integer"}
noise_type = 'trainable'  #@param ['zero', 'fixed', 'trainable']
optimizer = 'adam'  #@param ['sgd', 'adam','sgdm', 'adamax']
learning_rate = 0.4 #@param {type:"slider", min:0, max:1, step:0.05}
learning_rate_schedule = 'linear1cycledrop'  #@param ['fixed', 'linear1cycle', 'linear1cycledrop']
steps = 100 #@param {type:"slider", min:100, max:1000, step:50}



seed = abs(seed)
print('Estimated Runtime: {}s.\n'.format(round(0.23*steps)+6))

!python run.py \
  -seed $seed \
  -noise_type $noise_type \
  -opt_name $optimizer \
  -learning_rate $learning_rate \
  -steps $steps \
  -lr_schedule $learning_rate_schedule 


#@markdown ---
#@markdown *Αν δεις κάποιο error, ή αν δεν φορτώνει το browse κουμπάκι για να ανεβάσεις την εικόνα σου, ξανατρέξε αυτό το block.*

fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(mpimg.imread('/content/pulse/input/face.png'))
ax1.set_title('Original')
ax2.imshow(mpimg.imread('/content/pulse/runs/face.png'))
ax2.set_title('Result')
plt.show()

In [ ]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Κατέβασε σε υψηλή ανάλυση την εικόνα που δημιουργήθηκε</font></b>
try: files.download('/content/pulse/runs/face.png')
except: raise Exception("No result image")